In [90]:
from deepcave import Recorder, Objective
from deepcave.runs.converters.deepcave import DeepCAVERun
from deepcave.plugins.budget.budget_correlation import BudgetCorrelation
import pandas as pd
from arlbench.core.algorithms import DQN, PPO, SAC
from pathlib import Path
from ConfigSpace import ConfigurationSpace, Float, Categorical
import math
import os
import json

In [91]:
replacement_dict = {"atari_qbert": "Atari", 'atari_double_dunk': "Atari", 'atari_phoenix': "Atari", 'atari_this_game': "Atari", 'atari_battle_zone': "Atari", 
                    'box2d_lunar_lander': "Box2d", "box2d_bipedal_walker": "Box2d", 'cc_acrobot': "CC", 'cc_cartpole': "CC", 'cc_mountain_car': "CC", "cc_pendulum": "CC", 'cc_continuous_mountain_car': "CC",
                    'minigrid_door_key': "XLand", 'minigrid_empty_random': "XLand", 'minigrid_four_rooms': "XLand", 
                    'minigrid_unlock': "XLand", "brax_ant": "Brax", "brax_halfcheetah": "Brax", "brax_hopper": "Brax", "brax_humanoid": "Brax"}
algorithms = {"dqn": DQN, "ppo": PPO, "sac": SAC}
envs = ["atari_qbert", 'atari_double_dunk', 'atari_phoenix', 'atari_this_game', 'atari_battle_zone', 
        'box2d_lunar_lander', 'box2d_bipedal_walker', 'cc_acrobot', 'cc_cartpole', 'cc_mountain_car', 'cc_pendulum', 'cc_continuous_mountain_car',
        'minigrid_door_key', 'minigrid_empty_random', 'minigrid_four_rooms', 'minigrid_unlock', 'brax_ant', 'brax_halfcheetah', 'brax_hopper', 'brax_humanoid']
full_env_names =["CartPole-v1", "MountainCar-v0", "MountainCarContinuous-v0", "Acrobot-v1", "Pendulum-v1", "LunarLander-v2", "LunarLanderContinuous-v2", "BipedalWalker-v3", "halfcheetah", "ant", "hopper", "humanoid", "Qbert-v5", "DoubleDunk-v5", "Phoenix-v5", "ThisGame-v5", "BattleZone-v5", "MiniGrid-DoorKey-5x5", "MiniGrid-Empty-Random-5x5", "MiniGrid-FourRooms", "MiniGrid-Unlock"]
map_env_to_full = {"atari_qbert": "Qbert-v5", 'atari_double_dunk': "DoubleDunk-v5", 'atari_phoenix': "Phoenix-v5", 'atari_this_game': "NameThisGame-v5", 'atari_battle_zone': "BattleZone-v5", 
                   "box2d_lunar_lander": "LunarLander-v2", "box2d_bipedal_walker": "BipedalWalker-v3", 'cc_acrobot': "Acrobot-v1", 'cc_cartpole': "CartPole-v1", 'cc_mountain_car': "MountainCar-v0", "cc_pendulum": "Pendulum-v1", 'cc_continuous_mountain_car': "MountainCarContinuous-v0",
                   "minigrid_door_key": "MiniGrid-DoorKey-5x5", "minigrid_empty_random": "MiniGrid-EmptyRandom-5x5", "minigrid_four_rooms": "MiniGrid-FourRooms",
                         "minigrid_unlock": "MiniGrid-Unlock", "brax_ant": "ant", "brax_halfcheetah": "halfcheetah", "brax_hopper": "hopper", "brax_humanoid": "humanoid"}

In [92]:
def data_to_dynamic_deepcave(algorithm, env, domain=False, save_path=None):
    target_dir = f"../results_finished/sobol_landscaping/{domain.split('_')[0]}/{algorithm}/{algorithm}_{env}"
    target_path = f"../results_combined/sobol/{domain}_{algorithm}.csv"
    try:
        all_configs = pd.read_csv(target_path)
    except:
        print(f"Could not read {target_path}")
        return

    default_configspace = algorithms[algorithm].get_hpo_search_space().get_hyperparameter_names()
    configspace = ConfigurationSpace()
    for c in default_configspace:
        if c in ["buffer_prio_sampling", "use_target_network", "alpha_auto", "normalize_advantage"]:
            hp = Categorical(c, [True, False])
            configspace.add_hyperparameter(hp)
        else:
            key = [a for a in all_configs.keys() if c in a]
            if len(key) > 0:
                key = key[0]
                hp_min = all_configs[key].min()
                hp_max = all_configs[key].max()
                bounds = (min(0, hp_min-0.1*hp_min), hp_max+0.1*hp_max)
                hp = Float(c, bounds=bounds)
                configspace.add_hyperparameter(hp)
    default = configspace.get_default_configuration()
    accuracy = Objective("reward_mean", lower=min(all_configs["performance"]), upper=max(all_configs["performance"]), optimize="upper")
    save_path = save_path if save_path else f"deepcave_logs_curves/{algorithm}_{domain}"

    with Recorder(configspace, objectives=[accuracy], save_path=save_path) as r:
        for index, d in all_configs.iterrows():
            current_hps = d
            config = default
            for c in current_hps.keys():
                key = c.split(".")[-1]
                if key in config.keys():
                    if math.isnan(current_hps[c]):
                        current_hps[c] = 0
                    try:
                        value = float(current_hps[c])
                    except:
                        current_hps[c]
                    config[key] = value

            try:
                performance_file = f"{target_dir}/{d['seed']}/{d['run_id']}/evaluation.csv"
                partials = pd.read_csv(performance_file)
                partials["seed"] = d['seed']
                partials["returns"] = -1*partials["returns"]

                for i, p in partials.iterrows():
                    budget = p["steps"]/partials["steps"].max()
                    r.start(config=config, budget=budget, seed=p["seed"])
                    r.end(config=config, costs=p["returns"], budget=budget, seed=p["seed"])
            except:
                pass
            r.start(config=config, budget=1, seed=d["seed"])
            r.end(config=config, costs=current_hps["performance"], budget=1, seed=d["seed"])
    return save_path

In [93]:
def get_budget_correlations(path):
    # Instantiate the run
    run = DeepCAVERun.from_path(Path(path))
    objective_id = run.get_objective_ids()[0]
    budget_id = run.get_budget_ids()

    # Instantiate the plugin
    plugin = BudgetCorrelation()

    inputs = plugin.generate_inputs(
        objective_id=objective_id,
    )
    # Note: Filter variables are not considered.
    outputs = plugin.generate_outputs(run, inputs)

    # Finally, you can load the figure. Here, the filter variables play a role.
    # Alternatively: Use the matplotlib output (`load_mpl_outputs`) if available.
    figure = plugin.load_outputs(run, inputs, outputs)  # plotly.go figure
    return figure

In [95]:
deepcave_paths = []
algorithm = "ppo"
for e in envs:
    if not os.path.exists(f"deepcave_logs_curves/{algorithm}_{e}/run_1"):
        try:
            data_path = data_to_dynamic_deepcave(algorithm, map_env_to_full[e], domain=e)
            deepcave_paths.append(data_path)
        except Exception as e:
            print(e)
    else:
        deepcave_paths.append(f"deepcave_logs_curves/{algorithm}_{e}")

[Errno 2] No such file or directory: '../results_finished/sobol_landscaping/atari/ppo/ppo_BattleZone-v5/5/124/evaluation.csv'
Could not read ../results_combined/sobol/box2d_bipedal_walker_ppo.csv


In [96]:
corrs = {}

In [97]:
for data_path in deepcave_paths:
    try:
        img = get_budget_correlations(Path(data_path) / "run_1")
        img[1].write_image(data_path + "/budget_correlations.png")
        print(data_path + ": " + img[0])
        corrs[data_path] = img[0]
    except Exception as e:
        #print("Error")
        #print(e)
        pass

deepcave_logs_curves/ppo_atari_qbert: The budget correlation of (0.1, 0.2), (0.1, 0.3), (0.2, 0.3), (0.1, 0.4), (0.2, 0.4), (0.3, 0.4), (0.1, 0.5), (0.2, 0.5), (0.3, 0.5), (0.4, 0.5), (0.1, 0.6), (0.2, 0.6), (0.3, 0.6), (0.4, 0.6), (0.5, 0.6), (0.1, 0.7), (0.2, 0.7), (0.3, 0.7), (0.4, 0.7), (0.5, 0.7), (0.6, 0.7), (0.1, 0.8), (0.2, 0.8), (0.3, 0.8), (0.4, 0.8), (0.5, 0.8), (0.6, 0.8), (0.7, 0.8), (0.1, 0.9), (0.2, 0.9), (0.3, 0.9), (0.4, 0.9), (0.5, 0.9), (0.6, 0.9), (0.7, 0.9), (0.8, 0.9), (0.1, 1.0), (0.2, 1.0), (0.3, 1.0), (0.4, 1.0), (0.5, 1.0), (0.6, 1.0), (0.7, 1.0), (0.8, 1.0), (0.9, 1.0) is very strong.
deepcave_logs_curves/ppo_atari_double_dunk: The budget correlation of (0.1, 0.2), (0.2, 0.3), (0.2, 0.4), (0.3, 0.4), (0.2, 0.5), (0.3, 0.5), (0.4, 0.5), (0.3, 0.6), (0.4, 0.6), (0.5, 0.6), (0.2, 0.7), (0.3, 0.7), (0.4, 0.7), (0.5, 0.7), (0.6, 0.7), (0.2, 0.8), (0.3, 0.8), (0.4, 0.8), (0.5, 0.8), (0.7, 0.8), (0.2, 0.9), (0.3, 0.9), (0.6, 0.9), (0.7, 0.9), (0.2, 1.0), (0.3, 1.0),

In [98]:
json.dump(corrs, open(f"budget_correlations_{algorithm}.json", "w"))